In [1]:
import os
import sys
import cv2
import numpy as np
from pathlib import Path

# Configuración para notebooks - sin usar __file__
# Asume que estamos en el mismo directorio que main.py
BASE_DIR = os.getcwd()
sys.path.append(BASE_DIR)
print(f"Usando directorio base: {BASE_DIR}")

# Intenta importar los módulos necesarios
try:
    from models.vertex_detector import VertexDetector
    from utils.contorno import obtener_contorno_imagen
    print("Módulos importados correctamente")
except ImportError as e:
    print(f"Error importando módulos: {e}")
    print("Verifica que estás en el directorio correcto y que los módulos existen")
    # Salir o manejar el error

def determinar_orientacion_etiqueta(etiqueta_detection, image, corners):
    """
    Determina la orientación de la imagen basada en la ubicación y
    dimensiones de la etiqueta detectada.
    """
    # Extraer coordenadas de la caja contenedora de la etiqueta
    x1, y1, x2, y2 = etiqueta_detection['bbox']
    
    # Calcular el centro de la etiqueta
    centro_x = (x1 + x2) // 2
    centro_y = (y1 + y2) // 2
    
    # Calcular dimensiones de la etiqueta
    ancho_etiqueta = x2 - x1
    alto_etiqueta = y2 - y1
    
    # Determinar si la etiqueta tiene orientación horizontal (ancho > alto)
    es_horizontal = ancho_etiqueta > alto_etiqueta
    
    # Determinar el lado donde está la etiqueta respecto al centro de la palanquilla
    if corners is not None and len(corners) == 4:
        # Calcular centro de la palanquilla
        centro_palanquilla_x = sum(corner[0] for corner in corners) / 4
        centro_palanquilla_y = sum(corner[1] for corner in corners) / 4
        
        # Calcular distancias relativas
        dx = centro_x - centro_palanquilla_x
        dy = centro_y - centro_palanquilla_y
        
        # Determinar el lado predominante
        if abs(dx) > abs(dy):
            # La etiqueta está a la izquierda o derecha
            if dx < 0:
                lado = 'izquierda'
                # CORRECCIÓN: Siempre rotar 90° si está a la izquierda
                angulo = 90
            else:
                lado = 'derecha'
                # Si está a la derecha, rotar -90° si horizontal, 180° si vertical
                angulo = -90 if es_horizontal else 180
        else:
            # La etiqueta está arriba o abajo
            if dy < 0:
                lado = 'arriba'
                # Si está arriba, rotar 180° si horizontal, 90° si vertical
                angulo = 180 if es_horizontal else 90
            else:
                lado = 'abajo'
                # Si está abajo, no rotar si horizontal, -90° si vertical
                angulo = 0 if es_horizontal else -90
    else:
        # Si no hay vertices, usar solo las dimensiones de la etiqueta
        lado = 'desconocido'
        if es_horizontal:
            angulo = 0  # Asumir que ya está en la orientación correcta
        else:
            angulo = 90  # Rotar para que el lado más largo sea horizontal
    
    print(f"Etiqueta detectada en lado: {lado}, orientación: {'horizontal' if es_horizontal else 'vertical'}")
    print(f"Ángulo de rotación calculado: {angulo}°")
    
    return angulo, lado

# Función para procesar una imagen individual
def procesar_imagen(imagen_path, output_dir="pruebas_etiquetas"):
    """
    Procesa una sola imagen para probar la detección y rotación de etiquetas
    """
    # Crear directorio de salida
    os.makedirs(output_dir, exist_ok=True)
    
    # Configurar modelo - AJUSTA ESTA RUTA según donde tengas tu modelo
    vertex_model_path = r"D:\Trabajo modelos\PACC\YOLOv12 - copia\Models\Vertex\modelo_1.pt"
    print(f"Cargando modelo desde: {vertex_model_path}")
    
    # Intentar cargar el modelo
    try:
        vertex_detector = VertexDetector(vertex_model_path)
    except Exception as e:
        print(f"Error cargando el modelo: {e}")
        return
    
    # Cargar imagen
    image = cv2.imread(imagen_path)
    if image is None:
        print(f"Error: No se pudo cargar la imagen {imagen_path}")
        return
    
    # Nombre base de la imagen
    nombre_base = os.path.basename(imagen_path)
    nombre, ext = os.path.splitext(nombre_base)
    
    # Hacer una detección inicial de vértices
    try:
        print("\nDetectando contorno inicial de palanquilla...")
        initial_vertices, contorno_principal, _ = obtener_contorno_imagen(
            image, vertex_detector.model, vertex_detector.conf_threshold, 
            target_class=vertex_detector.target_class)
        print(f"Contorno detectado con {len(initial_vertices) if initial_vertices is not None else 0} vértices")
    except Exception as e:
        print(f"Error en detección inicial de vértices: {e}")
        initial_vertices = None
    
    # Detectar etiquetas
    print("\nBuscando etiquetas...")
    etiqueta_detections = []
    try:
        # Intentar un umbral de confianza más bajo para detectar mejor las etiquetas
        conf_threshold_etiqueta = 0.1  # Más bajo que el predeterminado
        vertex_result = vertex_detector.model.predict(
            image, conf=conf_threshold_etiqueta, device=vertex_detector.device)[0]
        
        if hasattr(vertex_detector.model, "names") and vertex_result.boxes is not None:
            class_names = vertex_detector.model.names
            print(f"Clases disponibles en el modelo: {class_names}")
            
            etiqueta_class_id = None
            # Buscar la clase "etiqueta"
            for id, name in class_names.items():
                if isinstance(name, str) and name.lower() == "etiqueta":
                    etiqueta_class_id = id
                    print(f"ID de clase para 'etiqueta' encontrado: {etiqueta_class_id}")
                    break
            
            # Si no encontramos "etiqueta", probar con class_0
            if etiqueta_class_id is None and 0 in class_names:
                etiqueta_class_id = 0
                print(f"No se encontró 'etiqueta' explícitamente. Usando class_0 como etiqueta, nombre: {class_names[0]}")
            
            if etiqueta_class_id is not None:
                boxes = vertex_result.boxes
                print(f"Total de detecciones: {len(boxes)}")
                
                for i, box in enumerate(boxes):
                    cls_id = int(box.cls[0].item())
                    conf = float(box.conf[0].item())
                    print(f"Detección #{i+1}: Clase {cls_id}, Confianza {conf:.2f}")
                    
                    if cls_id == etiqueta_class_id:
                        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
                        print(f"ETIQUETA detectada con confianza {conf:.2f} en bbox: ({x1}, {y1}, {x2}, {y2})")
                        
                        etiqueta_detections.append({
                            'bbox': (x1, y1, x2, y2),
                            'conf': conf,
                            'class': 'etiqueta',
                            'cls_id': cls_id
                        })
                
                print(f"Total etiquetas encontradas: {len(etiqueta_detections)}")
    except Exception as e:
        print(f"Error al buscar etiquetas: {e}")
        import traceback
        traceback.print_exc()
    
    # Crear imagen de visualización
    vis_img = image.copy()
    
    # Dibujar los vértices detectados
    if initial_vertices is not None:
        cv2.polylines(vis_img, [np.array(initial_vertices)], True, (0, 255, 0), 2)
        
    # Dibujar las etiquetas detectadas
    for i, etiqueta in enumerate(etiqueta_detections):
        x1, y1, x2, y2 = etiqueta['bbox']
        cv2.rectangle(vis_img, (x1, y1), (x2, y2), (0, 0, 255), 2)
        cv2.putText(vis_img, f"Etiqueta {i+1}", (x1, y1-10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    
    # Guardar imagen con detecciones
    detections_path = os.path.join(output_dir, f"{nombre}_detecciones{ext}")
    cv2.imwrite(detections_path, vis_img)
    print(f"Imagen con detecciones guardada en: {detections_path}")
    
    # Si se encontraron etiquetas, procesar rotación
    if etiqueta_detections:
        # Ordenar etiquetas por confianza
        etiqueta_detections.sort(key=lambda x: x['conf'], reverse=True)
        mejor_etiqueta = etiqueta_detections[0]
        
        # Determinar ángulo de rotación
        angulo_rotacion, lado_etiqueta = determinar_orientacion_etiqueta(
            mejor_etiqueta, image, initial_vertices)
        
        # Visualizar información de la etiqueta
        etiqueta_img = image.copy()
        x1, y1, x2, y2 = mejor_etiqueta['bbox']
        
        # Dibujar la etiqueta principal
        cv2.rectangle(etiqueta_img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
        # Dibujar información sobre la etiqueta
        ancho = x2 - x1
        alto = y2 - y1
        es_horizontal = ancho > alto
        
        info_text = [
            f"Lado: {lado_etiqueta}",
            f"Orientacion: {'Horizontal' if es_horizontal else 'Vertical'}",
            f"Ancho x Alto: {ancho} x {alto}",
            f"Rotacion: {angulo_rotacion} grados"
        ]
        
        for i, text in enumerate(info_text):
            cv2.putText(etiqueta_img, text, (10, 30 + 30*i), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        
        # Guardar imagen con info de etiqueta
        etiqueta_path = os.path.join(output_dir, f"{nombre}_info_etiqueta{ext}")
        cv2.imwrite(etiqueta_path, etiqueta_img)
        print(f"Imagen con info de etiqueta guardada en: {etiqueta_path}")
        
        # Aplicar rotación
        if angulo_rotacion != 0:
            h, w = image.shape[:2]
            center = (w // 2, h // 2)
            rotation_matrix = cv2.getRotationMatrix2D(center, angulo_rotacion, 1.0)
            rotated_image = cv2.warpAffine(image, rotation_matrix, (w, h), flags=cv2.INTER_CUBIC)
            
            # Guardar imagen rotada
            rotated_path = os.path.join(output_dir, f"{nombre}_rotada{ext}")
            cv2.imwrite(rotated_path, rotated_image)
            print(f"Imagen rotada guardada en: {rotated_path}")
            
            # Crear imagen comparativa lado a lado
            h_comp = min(h, 800)
            w_comp = int(w * (h_comp / h))
            
            original_resized = cv2.resize(image, (w_comp, h_comp))
            rotada_resized = cv2.resize(rotated_image, (w_comp, h_comp))
            
            # Comparación lado a lado
            comparison = np.zeros((h_comp, w_comp*2, 3), dtype=np.uint8)
            comparison[:, :w_comp] = original_resized
            comparison[:, w_comp:] = rotada_resized
            
            # Añadir etiquetas
            cv2.putText(comparison, "ORIGINAL", (10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(comparison, f"ROTADA {angulo_rotacion}°", (w_comp+10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            
            # Guardar comparación
            comp_path = os.path.join(output_dir, f"{nombre}_comparacion{ext}")
            cv2.imwrite(comp_path, comparison)
            print(f"Imagen comparativa guardada en: {comp_path}")
    else:
        print("No se detectaron etiquetas en la imagen.")

# Función para procesar un directorio
def procesar_directorio(directorio_path, output_dir=None):
    """
    Procesa todas las imágenes en un directorio
    """
    if output_dir is None:
        output_dir = os.path.join(os.getcwd(), "resultados_etiquetas")
    
    os.makedirs(output_dir, exist_ok=True)
    print(f"Directorio de salida: {output_dir}")
    
    # Extensiones válidas
    valid_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff']
    
    # Encontrar imágenes
    image_files = []
    for ext in valid_extensions:
        image_files.extend(list(Path(directorio_path).glob(f'*{ext}')))
        image_files.extend(list(Path(directorio_path).glob(f'*{ext.upper()}')))
    
    print(f"Encontradas {len(image_files)} imágenes en {directorio_path}")
    
    # Procesar cada imagen
    for i, img_path in enumerate(image_files):
        print(f"\n[{i+1}/{len(image_files)}] Procesando: {img_path.name}")
        
        # Crear carpeta específica para esta imagen
        img_output_dir = os.path.join(output_dir, img_path.stem)
        os.makedirs(img_output_dir, exist_ok=True)
        
        # Procesar
        procesar_imagen(str(img_path), img_output_dir)
    
    print("\nProcesamiento completado.")

directorio_path = r"D:\Trabajo modelos\PACC\YOLOv12 - copia\pruebas diagonales"
procesar_directorio(directorio_path)

Usando directorio base: d:\Trabajo modelos\PACC\YOLOv12 - copia\modulado
WARNING Error decoding JSON from C:\Users\locop\AppData\Roaming\Ultralytics\persistent_cache.json. Starting with an empty dictionary.
Módulos importados correctamente
Directorio de salida: d:\Trabajo modelos\PACC\YOLOv12 - copia\modulado\resultados_etiquetas
Encontradas 6 imágenes en D:\Trabajo modelos\PACC\YOLOv12 - copia\pruebas diagonales

[1/6] Procesando: 1era_imagen_png.rf.bd7bc392b4bb48111f1fbac79c898070.jpg
Cargando modelo desde: D:\Trabajo modelos\PACC\YOLOv12 - copia\Models\Vertex\modelo_1.pt
Buscando archivo YAML en: D:\Trabajo modelos\PACC\YOLOv12 - copia\Models\Vertex\modelo_1.yaml
Clases cargadas desde D:\Trabajo modelos\PACC\YOLOv12 - copia\Models\Vertex\modelo_1.yaml: ['etiqueta', 'palanquilla']
Clase objetivo 'palanquilla' encontrada en índice 1
Usando dispositivo: cuda:0
Modelo YOLO para detección de vértices cargado desde: D:\Trabajo modelos\PACC\YOLOv12 - copia\Models\Vertex\modelo_1.pt
Clase o

In [3]:
import json
import openpyxl
import os
import boto3
import io
import base64
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side
from openpyxl.utils import get_column_letter
from openpyxl.drawing.image import Image
from PIL import Image as PILImage

def lambda_handler(event, context):
    try:
        # Si los datos vienen en el evento, usarlos; de lo contrario usar datos de ejemplo
        if 'body' in event and event['body']:
            data = json.loads(event['body'])
        else:
            # Usar los mismos datos de ejemplo que estaban en el código original
            json_data = """{

                "general": {

                "seccion": "160x160 mm",

                "fecha": "8-4-2025",

                "calidad": "1105-J1/B",

                "nombre": "BUZA SUMERGIDA VD"

                },

                "defectos": [

                {

                "nombre": "Grieta Diagonal",

                "parametros": {

                "longitud_mm": 15,

                "distancia_a_superficie_mm": 15,

                "espesor_mm": 0.2

                },

                "nivel_criterio": [

                {"nivel": 0,"L_mm_max": 5,"D_mm_min": 10,"e_mm_min": 0.2},

                {"nivel": 1,"L_mm_max": 10,"D_mm_min": 5,"e_mm_min": 0.2},

                {"nivel": 2,"L_mm_max": 15,"D_mm_min": 3,"e_mm_min": 0.2},

                {"nivel": 3,"L_mm_gt": 15,"D_mm_lt": 3,"e_mm_lt": 0.2}

                ],

                "imagen_ruta": "s3://modelo-deteccion-pacc/imagenes-palanquillas/2025/4/8/BUZA SUMERGIDA VD/palanquilla-1744162008000.jpeg",

                "codigo": "B00172282",

                "calidad": "1105-J1/B",

                "linea": "L3",

                "unidad_medida": "mm"

                },

                {

                "nombre": "Grieta Medio Camino",

                "parametros": {

                "longitud_mm": 15,

                "distancia_a_superficie_mm": 15,

                "espesor_mm": 0.3

                },

                "nivel_criterio": [

                {"nivel": 0,"L_mm_max": 5,"D_mm_min": 10,"e_mm_min": 0.2},

                {"nivel": 1,"L_mm_max": 10,"D_mm_min": 5,"e_mm_min": 0.2},

                {"nivel": 2,"L_mm_max": 15,"D_mm_min": 3,"e_mm_min": 0.2},

                {"nivel": 3,"L_mm_gt": 15,"D_mm_lt": 3,"e_mm_lt": 0.2}

                ],

                "imagen_ruta": "s3://modelo-deteccion-pacc/imagenes-palanquillas/2025/4/8/BUZA SUMERGIDA VD/palanquilla-1744162008000.jpeg",

                "codigo": "B00172283",

                "calidad": "1105-J1/B",

                "linea": "L4",

                "unidad_medida": "mm"

                },

                {

                "nombre": "Grieta Medio Camino 2",

                "parametros": {

                "longitud_mm": 15,

                "distancia_a_superficie_mm": 15,

                "espesor_mm": 0.3

                },

                "nivel_criterio": [

                {"nivel": 0,"L_mm_max": 5,"D_mm_min": 10,"e_mm_min": 0.2},

                {"nivel": 1,"L_mm_max": 10,"D_mm_min": 5,"e_mm_min": 0.2},

                {"nivel": 2,"L_mm_max": 15,"D_mm_min": 3,"e_mm_min": 0.2},

                {"nivel": 3,"L_mm_gt": 15,"D_mm_lt": 3,"e_mm_lt": 0.2}

                ],

                "imagen_ruta": "s3://modelo-deteccion-pacc/imagenes-palanquillas/2025/4/8/BUZA SUMERGIDA VD/palanquilla-1744162008000.jpeg",

                "codigo": "B00172283",

                "calidad": "1105-J1/B",

                "linea": "L4",

                "unidad_medida": "mm"

                }

                ]

                }"""
            data = json.loads(json_data)
        
        # Crear un libro de Excel en memoria
        wb = openpyxl.Workbook()
        ws = wb.active
        
        # Configurar estilos
        title_font = Font(name='Arial', bold=True, size=12)
        header_font = Font(name='Arial', bold=True, size=10)
        normal_font = Font(name='Arial', size=10)
        blue_font = Font(name='Arial', bold=True, size=10, color='0000FF')
        red_font = Font(name='Arial', bold=True, size=10, color='FF0000')
        red_fill = PatternFill(start_color='FF0000', end_color='FF0000', fill_type='solid')
        dark_blue_fill = PatternFill(start_color='000080', end_color='000080', fill_type='solid')
        thin_border = Border(
            left=Side(style='thin'),
            right=Side(style='thin'),
            top=Side(style='thin'),
            bottom=Side(style='thin')
        )
        
        # Formato del título (Fila 1)
        ws.merge_cells('A1:L1')
        ws['A1'] = 'LECTURA DE MACROS CALIDAD' + data['general']['calidad'] + '(' + data['general']['nombre'] + ')'
        ws['A1'].font = title_font
        ws['A1'].alignment = Alignment(horizontal='center')
        ws['A1'].border = thin_border
        
        # Formato de la sección y campaña (Fila 2)
        ws['D2'] = 'SECCION:'
        ws['D2'].font = header_font
        ws['D2'].alignment = Alignment(horizontal='right')
        ws['D2'].border = thin_border
        
        ws['E2'] = data['general']['seccion']
        ws['E2'].font = blue_font
        ws['E2'].alignment = Alignment(horizontal='center')
        ws['E2'].border = thin_border
        
        ws.merge_cells('F2:G2')
        ws['F2'] = '160*160'
        ws['F2'].font = Font(bold=True, color='FFFFFF')
        ws['F2'].fill = red_fill
        ws['F2'].alignment = Alignment(horizontal='center')
        ws['F2'].border = thin_border
        ws['G2'].border = thin_border
        
        ws.merge_cells('H2:I2')
        ws['H2'] = f"CAMPAÑA: {data['general']['fecha']}"
        ws['H2'].font = header_font
        ws['H2'].alignment = Alignment(horizontal='left')
        ws['H2'].border = thin_border
        ws['I2'].border = thin_border
        
        # Aplicar bordes solo a las celdas que se utilizan
        used_cells = []
        
        # Función para generar las columnas para cada defecto
        def agregar_defecto(index, defecto, columna_inicial, fila_inicial):
            # Defectos información
            col1 = columna_inicial
            col2 = columna_inicial + 1
            col3 = columna_inicial + 2
            
            # Código, calidad y línea
            ws[f'{get_column_letter(col1)}{fila_inicial}'] = defecto['codigo']
            ws[f'{get_column_letter(col1)}{fila_inicial}'].font = red_font
            ws[f'{get_column_letter(col1)}{fila_inicial}'].alignment = Alignment(horizontal='center')
            used_cells.append(f'{get_column_letter(col1)}{fila_inicial}')
            
            ws[f'{get_column_letter(col2)}{fila_inicial}'] = defecto['calidad']
            ws[f'{get_column_letter(col2)}{fila_inicial}'].alignment = Alignment(horizontal='center')
            used_cells.append(f'{get_column_letter(col2)}{fila_inicial}')
            
            ws[f'{get_column_letter(col3)}{fila_inicial}'] = defecto['linea']
            ws[f'{get_column_letter(col3)}{fila_inicial}'].font = header_font
            ws[f'{get_column_letter(col3)}{fila_inicial}'].alignment = Alignment(horizontal='center')
            used_cells.append(f'{get_column_letter(col3)}{fila_inicial}')
            
            # Imagen (si está disponible en S3)
            if 'imagen_ruta' in defecto and defecto['imagen_ruta'].startswith('s3://'):
                try:
                    # Extraer detalles del path de S3
                    s3_path = defecto['imagen_ruta'][5:]  # eliminar 's3://'
                    bucket_name = s3_path.split('/')[0]
                    object_key = '/'.join(s3_path.split('/')[1:])
                    
                    # Conectar a S3 y descargar la imagen
                    s3 = boto3.client('s3')
                    response = s3.get_object(Bucket=bucket_name, Key=object_key)
                    img_data = response['Body'].read()
                    
                    # Método con PIL para preparar la imagen
                    img_file = io.BytesIO(img_data)
                    pil_img = PILImage.open(img_file)
                    
                    # Convertir a formato compatible si es necesario
                    if pil_img.format != 'PNG':
                        png_buffer = io.BytesIO()
                        pil_img = pil_img.convert('RGB')
                        pil_img.save(png_buffer, format='PNG')
                        png_buffer.seek(0)
                        
                        # Guardar en archivo temporal para que openpyxl pueda manejarlo
                        temp_img_path = f"/tmp/image_{defecto['codigo']}.png"
                        with open(temp_img_path, 'wb') as f:
                            f.write(png_buffer.getvalue())
                    else:
                        # Ya es PNG, guardar directamente
                        temp_img_path = f"/tmp/image_{defecto['codigo']}.png"
                        with open(temp_img_path, 'wb') as f:
                            f.write(img_data)
                    
                    # Usar el archivo temporal con openpyxl
                    img = Image(temp_img_path)
                    
                    # Cálculos para el tamaño
                    column_width_pixels = 50
                    row_height_pixels = 20
                    
                    # Definir anchura para cubrir 3 columnas
                    img.width = int(6.3 * column_width_pixels)
                    # Definir altura para 9 filas
                    img.height = int(10.4 * row_height_pixels)
                    
                    # Posicionar la imagen
                    ws.add_image(img, f'{get_column_letter(col1)}{fila_inicial+1}')
                    
                    # Registrar en el log
                    print(f"Imagen añadida desde {defecto['imagen_ruta']} usando PIL")
                    
                except Exception as e:
                    print(f"Error al añadir la imagen {index}: {str(e)}")
                    # No intentamos un método alternativo porque ya estamos usando PIL
            
            # Información de dimensiones
            fila_dims = fila_inicial + 9  # 9 filas después de la fila inicial para las dimensiones
            ws[f'{get_column_letter(col1)}{fila_dims}'] = 'Longitud: ' + str(defecto['parametros']['longitud_mm'])
            ws[f'{get_column_letter(col1)}{fila_dims}'].alignment = Alignment(horizontal='center')
            used_cells.append(f'{get_column_letter(col1)}{fila_dims}')
            
            ws[f'{get_column_letter(col2)}{fila_dims}'] = 'Distancia: ' + str(defecto['parametros']['distancia_a_superficie_mm'])
            ws[f'{get_column_letter(col2)}{fila_dims}'].alignment = Alignment(horizontal='center')
            used_cells.append(f'{get_column_letter(col2)}{fila_dims}')
            
            ws[f'{get_column_letter(col3)}{fila_dims}'] = 'Espesor: ' + str(defecto['parametros']['espesor_mm'])
            ws[f'{get_column_letter(col3)}{fila_dims}'].alignment = Alignment(horizontal='center')
            used_cells.append(f'{get_column_letter(col3)}{fila_dims}')
            
            return fila_dims + 3  # Retorna la próxima fila disponible (con un espacio de 3 filas)
        
        def agregar_abombamiento(col_inicial, fila_inicial, lado="izquierda"):
            col1 = col_inicial
            col2 = col_inicial + 1
            col1_letra = get_column_letter(col1)
            col2_letra = get_column_letter(col2)
            
            # ABOMBAMIENTO
            ws.merge_cells(f'{col1_letra}{fila_inicial}:{col2_letra}{fila_inicial}')
            ws[f'{col1_letra}{fila_inicial}'] = 'ABOMBAMIENTO'
            ws[f'{col1_letra}{fila_inicial}'].font = Font(name='Arial', bold=True, size=10)
            ws[f'{col1_letra}{fila_inicial}'].alignment = Alignment(horizontal='left')
            used_cells.append(f'{col1_letra}{fila_inicial}')
            used_cells.append(f'{col2_letra}{fila_inicial}')
            
            # I=
            fila = fila_inicial + 1
            ws.merge_cells(f'{col1_letra}{fila}:{col2_letra}{fila}')
            ws[f'{col1_letra}{fila}'] = 'I= 0'
            ws[f'{col1_letra}{fila}'].alignment = Alignment(horizontal='left')
            used_cells.append(f'{col1_letra}{fila}')
            used_cells.append(f'{col2_letra}{fila}')
            
            # M=
            fila = fila_inicial + 2
            ws.merge_cells(f'{col1_letra}{fila}:{col2_letra}{fila}')
            ws[f'{col1_letra}{fila}'] = 'M= 0'
            ws[f'{col1_letra}{fila}'].alignment = Alignment(horizontal='left')
            used_cells.append(f'{col1_letra}{fila}')
            used_cells.append(f'{col2_letra}{fila}')
            
            # T=
            fila = fila_inicial + 3
            ws.merge_cells(f'{col1_letra}{fila}:{col2_letra}{fila}')
            ws[f'{col1_letra}{fila}'] = 'T= 0'
            ws[f'{col1_letra}{fila}'].alignment = Alignment(horizontal='left')
            used_cells.append(f'{col1_letra}{fila}')
            used_cells.append(f'{col2_letra}{fila}')
            
            # ROMBO
            fila = fila_inicial + 4
            ws[f'{col1_letra}{fila}'] = 'ROMBO'
            ws[f'{col1_letra}{fila}'].font = Font(name='Arial', bold=True, size=10)
            ws[f'{col1_letra}{fila}'].alignment = Alignment(horizontal='left')
            used_cells.append(f'{col1_letra}{fila}')
            
            ws[f'{col2_letra}{fila}'] = '%'
            ws[f'{col2_letra}{fila}'].alignment = Alignment(horizontal='center')
            used_cells.append(f'{col2_letra}{fila}')
            
            # Valores del rombo
            fila = fila_inicial + 5
            ws[f'{col1_letra}{fila}'] = '0'
            ws[f'{col1_letra}{fila}'].font = blue_font
            ws[f'{col1_letra}{fila}'].alignment = Alignment(horizontal='center')
            used_cells.append(f'{col1_letra}{fila}')
            
            ws[f'{col2_letra}{fila}'] = '0'
            ws[f'{col2_letra}{fila}'].font = blue_font
            ws[f'{col2_letra}{fila}'].alignment = Alignment(horizontal='center')
            used_cells.append(f'{col2_letra}{fila}')
        
        # Organizar defectos (2 por fila)
        num_defectos = len(data['defectos'])
        fila_actual = 4  # Fila inicial para los defectos
        
        for i in range(0, num_defectos, 2):
            # Primer defecto de la fila
            if i < num_defectos:
                # Abombamiento izquierdo
                agregar_abombamiento(1, fila_actual, "izquierda")
                # Defecto izquierdo
                agregar_defecto(i, data['defectos'][i], 3, fila_actual)
            
            # Segundo defecto de la fila (si existe)
            if i + 1 < num_defectos:
                # Defecto derecho
                agregar_defecto(i + 1, data['defectos'][i + 1], 8, fila_actual)
                # Abombamiento derecho
                agregar_abombamiento(11, fila_actual, "derecha")
            
            # Actualizar la fila actual (cada defecto ocupa 12 filas de alto)
            fila_actual += 11
        
        # Configurar el ancho de las columnas - 15 columnas en total (2 defectos por fila)
        column_widths = {}
        for i in range(1, 13):
            if i in [1, 2, 11, 12]:  # Columnas de ABOMBAMIENTO y ROMBO
                column_widths[i] = 8
            else:
                column_widths[i] = 15
        
        for col, width in column_widths.items():
            ws.column_dimensions[get_column_letter(col)].width = width
        
        # Configurar la altura de las filas para las imágenes
        for row in range(4, fila_actual):
            ws.row_dimensions[row].height = 20
        
        # Leyenda de defectos - centrar al final del documento
        legend_row = fila_actual
        legend_start_col = 5
        legend_end_col = 8
        start_letter = get_column_letter(legend_start_col)
        end_letter = get_column_letter(legend_end_col)
        
        ws.merge_cells(f'{start_letter}{legend_row}:{end_letter}{legend_row}')
        ws[f'{start_letter}{legend_row}'] = 'LEYENDA DE DEFECTOS'
        ws[f'{start_letter}{legend_row}'].font = Font(bold=True, color='FFFFFF')
        ws[f'{start_letter}{legend_row}'].fill = dark_blue_fill
        ws[f'{start_letter}{legend_row}'].alignment = Alignment(horizontal='center')
        used_cells.extend([f'{get_column_letter(col)}{legend_row}' for col in range(legend_start_col, legend_end_col + 1)])
        
        # Datos de la leyenda
        defect_codes = [
            ('R', 'RECHUPE'),
            ('N.E.', 'NUCLEO ESPONJOSO'),
            ('GTS', 'GRIETAS'),
            ('L', 'LONGITUD'),
            ('D', 'DISTANCIA'),
            ('G.C', 'GRIETAS CENTRAL')
        ]
        
        for i, (code, desc) in enumerate(defect_codes):
            row = legend_row + i + 1
            ws[f'{start_letter}{row}'] = code
            ws[f'{start_letter}{row}'].alignment = Alignment(horizontal='center')
            used_cells.append(f'{start_letter}{row}')
            
            mid_letter = get_column_letter(legend_start_col + 1)
            ws[f'{mid_letter}{row}'] = ':'
            ws[f'{mid_letter}{row}'].alignment = Alignment(horizontal='center')
            used_cells.append(f'{mid_letter}{row}')
            
            # Merge cells for the description
            merge_start = get_column_letter(legend_start_col + 2)
            merge_end = end_letter
            ws.merge_cells(f'{merge_start}{row}:{merge_end}{row}')
            ws[f'{merge_start}{row}'] = desc
            ws[f'{merge_start}{row}'].font = blue_font
            ws[f'{merge_start}{row}'].alignment = Alignment(horizontal='center')
            used_cells.append(f'{merge_start}{row}')
            used_cells.append(f'{merge_end}{row}')
        
        # Aplicar bordes solo a las celdas utilizadas
        for cell_ref in used_cells:
            ws[cell_ref].border = thin_border
        
        # Guardar el archivo en memoria en lugar de en disco
        output = io.BytesIO()
        wb.save(output)
        output.seek(0)
        
        # Construir la ruta correcta en S3 usando los datos del informe
        s3_bucket = "modelo-deteccion-pacc"
        s3_path = f"reportes-palanquillas/{data['general']['fecha'].split('-')[2]}/{data['general']['fecha'].split('-')[1]}/{data['general']['fecha'].split('-')[0]}/{data['general']['nombre']}/Lectura_Macros_Calidad_{data['general']['fecha']}.xlsx"
        
        # Guardar el archivo en S3
        s3 = boto3.client('s3')
        s3.put_object(
            Body=output.getvalue(),
            Bucket=s3_bucket,
            Key=s3_path,
            ContentType='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'
        )
        
        # Codificar el archivo Excel en base64 para incluirlo en la respuesta si se necesita
        excel_data = base64.b64encode(output.getvalue()).decode('utf-8')
        
        # Respuesta de la función Lambda
        return {
            'statusCode': 200,
            'headers': {
                'Content-Type': 'application/json'
            },
            'body': json.dumps({
                'mensaje': 'Archivo Excel generado correctamente',
                'ruta_s3': f"s3://{s3_bucket}/{s3_path}",
                'nota_importante': 'Las imágenes han sido incluidas desde S3, verifique que tenga permisos adecuados para visualizarlas'
            })
        }
        
    except Exception as e:
        print(f"Error: {str(e)}")
        return {
            'statusCode': 500,
            'body': json.dumps({'error': str(e)})
        }

ModuleNotFoundError: No module named 'boto3'

In [4]:
pip install boto3


   ---------------------------------------- 0.0/13.5 MB ? eta -:--:--
   -------------------- ------------------- 7.1/13.5 MB 62.0 MB/s eta 0:00:01
   ---------------------- ----------------- 7.6/13.5 MB 20.4 MB/s eta 0:00:01
   ---------------------------- ----------- 9.7/13.5 MB 16.3 MB/s eta 0:00:01
   ---------------------------------------- 13.5/13.5 MB 18.5 MB/s eta 0:00:00

   ---------- ----------------------------- 1/4 [botocore]
   ---------- ----------------------------- 1/4 [botocore]
   ---------- ----------------------------- 1/4 [botocore]
   ---------- ----------------------------- 1/4 [botocore]
   ---------- ----------------------------- 1/4 [botocore]
   ---------- ----------------------------- 1/4 [botocore]
   ---------- ----------------------------- 1/4 [botocore]
   ---------- ----------------------------- 1/4 [botocore]
   ---------- ----------------------------- 1/4 [botocore]
   ------------------------------ --------- 3/4 [boto3]
   --------------------------

In [ ]:
lambda_handler(None, None)

In [10]:
import json
import openpyxl
import os
import io
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side
from openpyxl.utils import get_column_letter
from openpyxl.drawing.image import Image
from PIL import Image as PILImage

def lambda_handler(event, context):
    try:
        # Si los datos vienen en el evento, usarlos; de lo contrario usar datos de ejemplo
        if 'body' in event and event['body']:
            data = json.loads(event['body'])
        else:
            # Usar los mismos datos de ejemplo que estaban en el código original
            json_data = """{

                "general": {

                "seccion": "160x160 mm",

                "fecha": "8-4-2025",

                "calidad": "1105-J1/B",

                "nombre": "BUZA SUMERGIDA VD"

                },

                "defectos": [

                {

                "nombre": "Grieta Diagonal",

                "parametros": {

                "longitud_mm": 15,

                "distancia_a_superficie_mm": 15,

                "espesor_mm": 0.2

                },

                "nivel_criterio": [

                {"nivel": 0,"L_mm_max": 5,"D_mm_min": 10,"e_mm_min": 0.2},

                {"nivel": 1,"L_mm_max": 10,"D_mm_min": 5,"e_mm_min": 0.2},

                {"nivel": 2,"L_mm_max": 15,"D_mm_min": 3,"e_mm_min": 0.2},

                {"nivel": 3,"L_mm_gt": 15,"D_mm_lt": 3,"e_mm_lt": 0.2}

                ],

                "imagen_ruta": "D:\\Trabajo modelos\\PACC\\YOLOv12 - copia\\pruebas diagonales\\DSC00002.JPG",

                "codigo": "B00172282",

                "calidad": "1105-J1/B",

                "linea": "L3",

                "unidad_medida": "mm"

                },

                {

                "nombre": "Grieta Medio Camino",

                "parametros": {

                "longitud_mm": 15,

                "distancia_a_superficie_mm": 15,

                "espesor_mm": 0.3

                },

                "nivel_criterio": [

                {"nivel": 0,"L_mm_max": 5,"D_mm_min": 10,"e_mm_min": 0.2},

                {"nivel": 1,"L_mm_max": 10,"D_mm_min": 5,"e_mm_min": 0.2},

                {"nivel": 2,"L_mm_max": 15,"D_mm_min": 3,"e_mm_min": 0.2},

                {"nivel": 3,"L_mm_gt": 15,"D_mm_lt": 3,"e_mm_lt": 0.2}

                ],

                "imagen_ruta": "D:\\Trabajo modelos\\PACC\\YOLOv12 - copia\\pruebas diagonales\\DSC00003.JPG",

                "codigo": "B00172283",

                "calidad": "1105-J1/B",

                "linea": "L4",

                "unidad_medida": "mm"

                },

                {

                "nombre": "Grieta Medio Camino 2",

                "parametros": {

                "longitud_mm": 15,

                "distancia_a_superficie_mm": 15,

                "espesor_mm": 0.3

                },

                "nivel_criterio": [

                {"nivel": 0,"L_mm_max": 5,"D_mm_min": 10,"e_mm_min": 0.2},

                {"nivel": 1,"L_mm_max": 10,"D_mm_min": 5,"e_mm_min": 0.2},

                {"nivel": 2,"L_mm_max": 15,"D_mm_min": 3,"e_mm_min": 0.2},

                {"nivel": 3,"L_mm_gt": 15,"D_mm_lt": 3,"e_mm_lt": 0.2}

                ],

                "imagen_ruta": "D:\\Trabajo modelos\\PACC\\YOLOv12 - copia\\pruebas diagonales\\DSC00004.JPG",

                "codigo": "B00172283",

                "calidad": "1105-J1/B",

                "linea": "L4",

                "unidad_medida": "mm"

                }

                ]

                }"""
            data = json.loads(json_data)
        
        # Crear un libro de Excel en memoria
        wb = openpyxl.Workbook()
        ws = wb.active
        
        # Configurar estilos
        title_font = Font(name='Arial', bold=True, size=12)
        header_font = Font(name='Arial', bold=True, size=10)
        normal_font = Font(name='Arial', size=10)
        blue_font = Font(name='Arial', bold=True, size=10, color='0000FF')
        red_font = Font(name='Arial', bold=True, size=10, color='FF0000')
        red_fill = PatternFill(start_color='FF0000', end_color='FF0000', fill_type='solid')
        dark_blue_fill = PatternFill(start_color='000080', end_color='000080', fill_type='solid')
        thin_border = Border(
            left=Side(style='thin'),
            right=Side(style='thin'),
            top=Side(style='thin'),
            bottom=Side(style='thin')
        )
        
        # Formato del título (Fila 1)
        ws.merge_cells('A1:L1')
        ws['A1'] = 'LECTURA DE MACROS CALIDAD' + data['general']['calidad'] + '(' + data['general']['nombre'] + ')'
        ws['A1'].font = title_font
        ws['A1'].alignment = Alignment(horizontal='center')
        ws['A1'].border = thin_border
        
        # Formato de la sección y campaña (Fila 2)
        ws['D2'] = 'SECCION:'
        ws['D2'].font = header_font
        ws['D2'].alignment = Alignment(horizontal='right')
        ws['D2'].border = thin_border
        
        ws['E2'] = data['general']['seccion']
        ws['E2'].font = blue_font
        ws['E2'].alignment = Alignment(horizontal='center')
        ws['E2'].border = thin_border
        
        ws.merge_cells('F2:G2')
        ws['F2'] = '160*160'
        ws['F2'].font = Font(bold=True, color='FFFFFF')
        ws['F2'].fill = red_fill
        ws['F2'].alignment = Alignment(horizontal='center')
        ws['F2'].border = thin_border
        ws['G2'].border = thin_border
        
        ws.merge_cells('H2:I2')
        ws['H2'] = f"CAMPAÑA: {data['general']['fecha']}"
        ws['H2'].font = header_font
        ws['H2'].alignment = Alignment(horizontal='left')
        ws['H2'].border = thin_border
        ws['I2'].border = thin_border
        
        # Aplicar bordes solo a las celdas que se utilizan
        used_cells = []
        
        # Función para generar las columnas para cada defecto
        def agregar_defecto(index, defecto, columna_inicial, fila_inicial):
            # Defectos información
            col1 = columna_inicial
            col2 = columna_inicial + 1
            col3 = columna_inicial + 2
            
            # Código, calidad y línea
            ws[f'{get_column_letter(col1)}{fila_inicial}'] = defecto['codigo']
            ws[f'{get_column_letter(col1)}{fila_inicial}'].font = red_font
            ws[f'{get_column_letter(col1)}{fila_inicial}'].alignment = Alignment(horizontal='center')
            used_cells.append(f'{get_column_letter(col1)}{fila_inicial}')
            
            ws[f'{get_column_letter(col2)}{fila_inicial}'] = defecto['calidad']
            ws[f'{get_column_letter(col2)}{fila_inicial}'].alignment = Alignment(horizontal='center')
            used_cells.append(f'{get_column_letter(col2)}{fila_inicial}')
            
            ws[f'{get_column_letter(col3)}{fila_inicial}'] = defecto['linea']
            ws[f'{get_column_letter(col3)}{fila_inicial}'].font = header_font
            ws[f'{get_column_letter(col3)}{fila_inicial}'].alignment = Alignment(horizontal='center')
            used_cells.append(f'{get_column_letter(col3)}{fila_inicial}')
            
            # Imagen (ahora desde archivo local)
            if 'imagen_ruta' in defecto and defecto['imagen_ruta']:
                try:
                    # Usar la ruta de imagen local directamente
                    img_path = defecto['imagen_ruta']
                    
                    # Verificar si el archivo existe
                    if os.path.exists(img_path):
                        # Método con PIL para preparar la imagen
                        pil_img = PILImage.open(img_path)
                        
                        # Convertir a formato compatible si es necesario
                        if pil_img.format != 'PNG':
                            png_buffer = io.BytesIO()
                            pil_img = pil_img.convert('RGB')
                            pil_img.save(png_buffer, format='PNG')
                            png_buffer.seek(0)
                            
                            # Guardar en archivo temporal para que openpyxl pueda manejarlo
                            temp_dir = os.path.join(os.getcwd(), "temp")
                            os.makedirs(temp_dir, exist_ok=True)
                            temp_img_path = os.path.join(temp_dir, f"image_{defecto['codigo']}.png")
                            with open(temp_img_path, 'wb') as f:
                                f.write(png_buffer.getvalue())
                        else:
                            # Ya es PNG, usar directamente
                            temp_img_path = img_path
                        
                        # Usar la imagen con openpyxl
                        img = Image(temp_img_path)
                        
                        # Cálculos para el tamaño
                        column_width_pixels = 50
                        row_height_pixels = 20
                        
                        # Definir anchura para cubrir 3 columnas
                        img.width = int(6.3 * column_width_pixels)
                        # Definir altura para 9 filas
                        img.height = int(10.4 * row_height_pixels)
                        
                        # Posicionar la imagen
                        ws.add_image(img, f'{get_column_letter(col1)}{fila_inicial+1}')
                        
                        # Registrar en el log
                        print(f"Imagen añadida desde {defecto['imagen_ruta']}")
                    else:
                        print(f"No se encontró el archivo de imagen: {img_path}")
                    
                except Exception as e:
                    print(f"Error al añadir la imagen {index}: {str(e)}")
            
            # Información de dimensiones
            fila_dims = fila_inicial + 9  # 9 filas después de la fila inicial para las dimensiones
            ws[f'{get_column_letter(col1)}{fila_dims}'] = 'Longitud: ' + str(defecto['parametros']['longitud_mm'])
            ws[f'{get_column_letter(col1)}{fila_dims}'].alignment = Alignment(horizontal='center')
            used_cells.append(f'{get_column_letter(col1)}{fila_dims}')
            
            ws[f'{get_column_letter(col2)}{fila_dims}'] = 'Distancia: ' + str(defecto['parametros']['distancia_a_superficie_mm'])
            ws[f'{get_column_letter(col2)}{fila_dims}'].alignment = Alignment(horizontal='center')
            used_cells.append(f'{get_column_letter(col2)}{fila_dims}')
            
            ws[f'{get_column_letter(col3)}{fila_dims}'] = 'Espesor: ' + str(defecto['parametros']['espesor_mm'])
            ws[f'{get_column_letter(col3)}{fila_dims}'].alignment = Alignment(horizontal='center')
            used_cells.append(f'{get_column_letter(col3)}{fila_dims}')
            
            return fila_dims + 3  # Retorna la próxima fila disponible (con un espacio de 3 filas)
        
        def agregar_abombamiento(col_inicial, fila_inicial, lado="izquierda"):
            col1 = col_inicial
            col2 = col_inicial + 1
            col1_letra = get_column_letter(col1)
            col2_letra = get_column_letter(col2)
            
            # ABOMBAMIENTO
            ws.merge_cells(f'{col1_letra}{fila_inicial}:{col2_letra}{fila_inicial}')
            ws[f'{col1_letra}{fila_inicial}'] = 'ABOMBAMIENTO'
            ws[f'{col1_letra}{fila_inicial}'].font = Font(name='Arial', bold=True, size=10)
            ws[f'{col1_letra}{fila_inicial}'].alignment = Alignment(horizontal='left')
            used_cells.append(f'{col1_letra}{fila_inicial}')
            used_cells.append(f'{col2_letra}{fila_inicial}')
            
            # I=
            fila = fila_inicial + 1
            ws.merge_cells(f'{col1_letra}{fila}:{col2_letra}{fila}')
            ws[f'{col1_letra}{fila}'] = 'I= 0'
            ws[f'{col1_letra}{fila}'].alignment = Alignment(horizontal='left')
            used_cells.append(f'{col1_letra}{fila}')
            used_cells.append(f'{col2_letra}{fila}')
            
            # M=
            fila = fila_inicial + 2
            ws.merge_cells(f'{col1_letra}{fila}:{col2_letra}{fila}')
            ws[f'{col1_letra}{fila}'] = 'M= 0'
            ws[f'{col1_letra}{fila}'].alignment = Alignment(horizontal='left')
            used_cells.append(f'{col1_letra}{fila}')
            used_cells.append(f'{col2_letra}{fila}')
            
            # T=
            fila = fila_inicial + 3
            ws.merge_cells(f'{col1_letra}{fila}:{col2_letra}{fila}')
            ws[f'{col1_letra}{fila}'] = 'T= 0'
            ws[f'{col1_letra}{fila}'].alignment = Alignment(horizontal='left')
            used_cells.append(f'{col1_letra}{fila}')
            used_cells.append(f'{col2_letra}{fila}')
            
            # ROMBO
            fila = fila_inicial + 4
            ws[f'{col1_letra}{fila}'] = 'ROMBO'
            ws[f'{col1_letra}{fila}'].font = Font(name='Arial', bold=True, size=10)
            ws[f'{col1_letra}{fila}'].alignment = Alignment(horizontal='left')
            used_cells.append(f'{col1_letra}{fila}')
            
            ws[f'{col2_letra}{fila}'] = '%'
            ws[f'{col2_letra}{fila}'].alignment = Alignment(horizontal='center')
            used_cells.append(f'{col2_letra}{fila}')
            
            # Valores del rombo
            fila = fila_inicial + 5
            ws[f'{col1_letra}{fila}'] = '0'
            ws[f'{col1_letra}{fila}'].font = blue_font
            ws[f'{col1_letra}{fila}'].alignment = Alignment(horizontal='center')
            used_cells.append(f'{col1_letra}{fila}')
            
            ws[f'{col2_letra}{fila}'] = '0'
            ws[f'{col2_letra}{fila}'].font = blue_font
            ws[f'{col2_letra}{fila}'].alignment = Alignment(horizontal='center')
            used_cells.append(f'{col2_letra}{fila}')
        
        # Organizar defectos (2 por fila)
        num_defectos = len(data['defectos'])
        fila_actual = 4  # Fila inicial para los defectos
        
        for i in range(0, num_defectos, 2):
            # Primer defecto de la fila
            if i < num_defectos:
                # Abombamiento izquierdo
                agregar_abombamiento(1, fila_actual, "izquierda")
                # Defecto izquierdo
                agregar_defecto(i, data['defectos'][i], 3, fila_actual)
            
            # Segundo defecto de la fila (si existe)
            if i + 1 < num_defectos:
                # Defecto derecho
                agregar_defecto(i + 1, data['defectos'][i + 1], 8, fila_actual)
                # Abombamiento derecho
                agregar_abombamiento(11, fila_actual, "derecha")
            
            # Actualizar la fila actual (cada defecto ocupa 12 filas de alto)
            fila_actual += 11
        
        # Configurar el ancho de las columnas - 15 columnas en total (2 defectos por fila)
        column_widths = {}
        for i in range(1, 13):
            if i in [1, 2, 11, 12]:  # Columnas de ABOMBAMIENTO y ROMBO
                column_widths[i] = 8
            else:
                column_widths[i] = 15
        
        for col, width in column_widths.items():
            ws.column_dimensions[get_column_letter(col)].width = width
        
        # Configurar la altura de las filas para las imágenes
        for row in range(4, fila_actual):
            ws.row_dimensions[row].height = 20
        
        # Leyenda de defectos - centrar al final del documento
        legend_row = fila_actual
        legend_start_col = 5
        legend_end_col = 8
        start_letter = get_column_letter(legend_start_col)
        end_letter = get_column_letter(legend_end_col)
        
        ws.merge_cells(f'{start_letter}{legend_row}:{end_letter}{legend_row}')
        ws[f'{start_letter}{legend_row}'] = 'LEYENDA DE DEFECTOS'
        ws[f'{start_letter}{legend_row}'].font = Font(bold=True, color='FFFFFF')
        ws[f'{start_letter}{legend_row}'].fill = dark_blue_fill
        ws[f'{start_letter}{legend_row}'].alignment = Alignment(horizontal='center')
        used_cells.extend([f'{get_column_letter(col)}{legend_row}' for col in range(legend_start_col, legend_end_col + 1)])
        
        # Datos de la leyenda
        defect_codes = [
            ('R', 'RECHUPE'),
            ('N.E.', 'NUCLEO ESPONJOSO'),
            ('GTS', 'GRIETAS'),
            ('L', 'LONGITUD'),
            ('D', 'DISTANCIA'),
            ('G.C', 'GRIETAS CENTRAL')
        ]
        
        for i, (code, desc) in enumerate(defect_codes):
            row = legend_row + i + 1
            ws[f'{start_letter}{row}'] = code
            ws[f'{start_letter}{row}'].alignment = Alignment(horizontal='center')
            used_cells.append(f'{start_letter}{row}')
            
            mid_letter = get_column_letter(legend_start_col + 1)
            ws[f'{mid_letter}{row}'] = ':'
            ws[f'{mid_letter}{row}'].alignment = Alignment(horizontal='center')
            used_cells.append(f'{mid_letter}{row}')
            
            # Merge cells for the description
            merge_start = get_column_letter(legend_start_col + 2)
            merge_end = end_letter
            ws.merge_cells(f'{merge_start}{row}:{merge_end}{row}')
            ws[f'{merge_start}{row}'] = desc
            ws[f'{merge_start}{row}'].font = blue_font
            ws[f'{merge_start}{row}'].alignment = Alignment(horizontal='center')
            used_cells.append(f'{merge_start}{row}')
            used_cells.append(f'{merge_end}{row}')
        
        # Aplicar bordes solo a las celdas utilizadas
        for cell_ref in used_cells:
            ws[cell_ref].border = thin_border
        
        # Definir ruta para guardar localmente
        # Creamos la estructura de directorios según la fecha y nombre
        output_dir = os.path.join(
            os.getcwd(),
            "reportes-palanquillas",
            data['general']['fecha'].split('-')[2],
            data['general']['fecha'].split('-')[1],
            data['general']['fecha'].split('-')[0],
            data['general']['nombre']
        )
        
        # Asegurarse de que exista el directorio
        os.makedirs(output_dir, exist_ok=True)
        
        # Ruta completa del archivo
        output_file = os.path.join(
            output_dir,
            f"Lectura_Macros_Calidad_{data['general']['fecha']}.xlsx"
        )
        
        # Guardar el archivo localmente
        wb.save(output_file)
        
        # Respuesta de la función
        return {
            'statusCode': 200,
            'headers': {
                'Content-Type': 'application/json'
            },
            'body': json.dumps({
                'mensaje': 'Archivo Excel generado correctamente',
                'ruta_local': output_file,
                'nota_importante': 'Las imágenes han sido incluidas desde archivos locales'
            })
        }
        
    except Exception as e:
        print(f"Error: {str(e)}")
        return {
            'statusCode': 500,
            'body': json.dumps({'error': str(e)})
        }

In [11]:
event = {
    "version": "2.0",
    "routeKey": "ANY /generate-excel",
    "rawPath": "/generate-excel",
    "rawQueryString": "",
    "headers": {
        "content-type": "application/json"
    },
    "requestContext": {
        "accountId": "123456789012",
        "apiId": "api-id",
        "domainName": "id.execute-api.region.amazonaws.com",
        "domainPrefix": "id",
        "http": {
            "method": "POST",
            "path": "/generate-excel",
            "protocol": "HTTP/1.1",
            "sourceIp": "127.0.0.1",
            "userAgent": "python-requests/2.25.1"
        },
        "requestId": "id",
        "routeKey": "ANY /generate-excel",
        "stage": "$default",
        "time": "10/May/2025:12:34:56 +0000",
        "timeEpoch": 1620651296000
    },
    "body": "{\"general\":{\"seccion\":\"160x160 mm\",\"fecha\":\"8-4-2025\",\"calidad\":\"1105-J1/B\",\"nombre\":\"BUZA SUMERGIDA VD\"},\"defectos\":[{\"nombre\":\"Grieta Diagonal\",\"parametros\":{\"longitud_mm\":15,\"distancia_a_superficie_mm\":15,\"espesor_mm\":0.2},\"nivel_criterio\":[{\"nivel\":0,\"L_mm_max\":5,\"D_mm_min\":10,\"e_mm_min\":0.2},{\"nivel\":1,\"L_mm_max\":10,\"D_mm_min\":5,\"e_mm_min\":0.2},{\"nivel\":2,\"L_mm_max\":15,\"D_mm_min\":3,\"e_mm_min\":0.2},{\"nivel\":3,\"L_mm_gt\":15,\"D_mm_lt\":3,\"e_mm_lt\":0.2}],\"imagen_ruta\":\"D:\\\\Trabajo modelos\\\\PACC\\\\YOLOv12 - copia\\\\pruebas diagonales\\\\DSC00002.JPG\",\"codigo\":\"B00172282\",\"calidad\":\"1105-J1/B\",\"linea\":\"L3\",\"unidad_medida\":\"mm\"},{\"nombre\":\"Grieta Transversal\",\"parametros\":{\"longitud_mm\":12,\"distancia_a_superficie_mm\":8,\"espesor_mm\":0.3},\"nivel_criterio\":[{\"nivel\":0,\"L_mm_max\":5,\"D_mm_min\":10,\"e_mm_min\":0.2},{\"nivel\":1,\"L_mm_max\":10,\"D_mm_min\":5,\"e_mm_min\":0.2},{\"nivel\":2,\"L_mm_max\":15,\"D_mm_min\":3,\"e_mm_min\":0.2},{\"nivel\":3,\"L_mm_gt\":15,\"D_mm_lt\":3,\"e_mm_lt\":0.2}],\"imagen_ruta\":\"D:\\\\Trabajo modelos\\\\PACC\\\\YOLOv12 - copia\\\\pruebas diagonales\\\\DSC00003.JPG\",\"codigo\":\"B00172283\",\"calidad\":\"1105-J1/B\",\"linea\":\"L4\",\"unidad_medida\":\"mm\"}]}",
    "isBase64Encoded": False
}

In [12]:
lambda_handler(event, None)

Imagen añadida desde D:\Trabajo modelos\PACC\YOLOv12 - copia\pruebas diagonales\DSC00002.JPG
Imagen añadida desde D:\Trabajo modelos\PACC\YOLOv12 - copia\pruebas diagonales\DSC00003.JPG


{'statusCode': 200,
 'headers': {'Content-Type': 'application/json'},
 'body': '{"mensaje": "Archivo Excel generado correctamente", "ruta_local": "d:\\\\Trabajo modelos\\\\PACC\\\\YOLOv12 - copia\\\\modulado\\\\reportes-palanquillas\\\\2025\\\\4\\\\8\\\\BUZA SUMERGIDA VD\\\\Lectura_Macros_Calidad_8-4-2025.xlsx", "nota_importante": "Las im\\u00e1genes han sido incluidas desde archivos locales"}'}